# Descarga y Analisis de Datos

Este notebook muestra como obtener distribuciones de datasets y analizar sus datos.

In [ ]:
import sys
sys.path.insert(0, '..')

import asyncio
import json
import pandas as pd
import matplotlib.pyplot as plt
import httpx
import io

from server import get_distributions, search_datasets

def get_fn(tool):
    return tool.fn if hasattr(tool, 'fn') else tool

# Configurar pandas para mostrar mas columnas
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

## 1. Obtener Distribuciones de un Dataset

In [ ]:
# Primero buscar un dataset con CSV
async def buscar_dataset_csv():
    fn = get_fn(search_datasets)
    result = await fn(title="poblacion", format="csv")
    return json.loads(result)

datasets = await buscar_dataset_csv()
print(f"Datasets encontrados: {datasets.get('total_in_page')}")

if datasets.get('datasets'):
    primer_dataset = datasets['datasets'][0]
    print(f"\nDataset: {primer_dataset.get('title')}")
    print(f"URI: {primer_dataset.get('uri')}")

In [ ]:
# Obtener distribuciones con preview
async def obtener_distribuciones(dataset_uri):
    dataset_id = dataset_uri.split('/')[-1]
    fn = get_fn(get_distributions)
    result = await fn(dataset_id=dataset_id, include_preview=True, preview_rows=5)
    return json.loads(result)

if datasets.get('datasets'):
    distribuciones = await obtener_distribuciones(primer_dataset.get('uri'))
    print(f"Distribuciones: {distribuciones.get('total_in_page')}")
    
    for dist in distribuciones.get('distributions', []):
        print(f"\n- Formato: {dist.get('format')}")
        print(f"  URL: {dist.get('access_url', '')[:80]}...")
        if dist.get('preview'):
            preview = dist['preview']
            print(f"  Columnas: {preview.get('columns', [])}")
            print(f"  Filas totales: {preview.get('total_rows')}")

## 2. Descargar y Cargar CSV en Pandas

In [ ]:
async def descargar_csv(url):
    """Descargar CSV y cargar en DataFrame."""
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.get(url)
        response.raise_for_status()
        
        # Detectar encoding
        content = response.content
        try:
            text = content.decode('utf-8')
        except UnicodeDecodeError:
            text = content.decode('latin-1')
        
        # Detectar delimitador
        first_line = text.split('\n')[0]
        delimiter = ';' if first_line.count(';') > first_line.count(',') else ','
        
        return pd.read_csv(io.StringIO(text), delimiter=delimiter)

# Buscar una distribucion CSV y descargarla
if 'distribuciones' in dir():
    for dist in distribuciones.get('distributions', []):
        if dist.get('format') and 'csv' in str(dist.get('format')).lower():
            url = dist.get('access_url')
            if url:
                try:
                    df = await descargar_csv(url)
                    print(f"Datos descargados: {len(df)} filas, {len(df.columns)} columnas")
                    display(df.head())
                    break
                except Exception as e:
                    print(f"Error descargando: {e}")

## 3. Analisis Basico

In [ ]:
if 'df' in dir() and not df.empty:
    print("Informacion del DataFrame:")
    print(df.info())
    
    print("\nEstadisticas:")
    display(df.describe())

## 4. Visualizacion

In [ ]:
if 'df' in dir() and not df.empty:
    # Buscar columnas numericas para graficar
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    
    if numeric_cols:
        fig, ax = plt.subplots(figsize=(10, 6))
        df[numeric_cols[0]].hist(ax=ax, bins=30)
        ax.set_title(f'Distribucion de {numeric_cols[0]}')
        ax.set_xlabel(numeric_cols[0])
        ax.set_ylabel('Frecuencia')
        plt.tight_layout()
        plt.show()

## 5. Ejemplo Completo: Analizar Datos de Poblacion

In [ ]:
async def ejemplo_poblacion():
    """Ejemplo completo de busqueda y analisis."""
    
    # 1. Buscar datasets de poblacion con CSV
    fn = get_fn(search_datasets)
    result = await fn(title="censo", format="csv")
    datasets = json.loads(result)
    
    if not datasets.get('datasets'):
        print("No se encontraron datasets")
        return None
    
    print(f"Encontrados {len(datasets['datasets'])} datasets")
    
    # 2. Obtener distribuciones del primero
    dataset = datasets['datasets'][0]
    dataset_id = dataset['uri'].split('/')[-1]
    
    fn_dist = get_fn(get_distributions)
    dist_result = await fn_dist(dataset_id=dataset_id)
    distribuciones = json.loads(dist_result)
    
    # 3. Encontrar CSV
    csv_url = None
    for dist in distribuciones.get('distributions', []):
        fmt = str(dist.get('format', '')).lower()
        if 'csv' in fmt:
            csv_url = dist.get('access_url')
            break
    
    if not csv_url:
        print("No se encontro CSV")
        return None
    
    # 4. Descargar y analizar
    df = await descargar_csv(csv_url)
    print(f"\nDataset: {dataset.get('title')}")
    print(f"Filas: {len(df)}, Columnas: {len(df.columns)}")
    
    return df

try:
    df_poblacion = await ejemplo_poblacion()
    if df_poblacion is not None:
        display(df_poblacion.head(10))
except Exception as e:
    print(f"Error en ejemplo: {e}")

## Siguiente: Integraciones

En el siguiente notebook veremos como usar las integraciones con INE, AEMET y BOE.

[04_integraciones.ipynb](04_integraciones.ipynb)